# 라이브러리 호출 및 access key

In [5]:
import requests
import json
import pprint
import pandas as pd
from itertools import islice

In [2]:
flight_access_key = '***'
price_access_key = '***'
price_secret_key = '***'

# 도시, 국가 list -> dict

In [3]:
offset = 0
limit = 100

## country

In [ ]:
country_set = set()

while True:
    url = f'http://api.aviationstack.com/v1/airports?access_key={access_key}&offset={offset}&limit={limit}'
    ctr_response = requests.get(url)
    data = json.loads(ctr_response.text)
    for d in data.get('data'):
        country_set.add(d.get('country_name'))
    if offset >= data['pagination']['total']:
        break
    offset += limit

country_set.remove(None)
print(len(country_set))

232


In [ ]:
country = [i.replace('\n', '').replace('\r', '') for i in country_set]
country[:10]

['Czech Republic',
 'Haiti',
 'Kyrgyzstan',
 'Central African Republic',
 'Sweden',
 'Bolivia',
 'Nauru',
 'Honduras',
 'Jersey',
 'Niue']

## city & airport

In [ ]:
offset = 0

airport_set = set()
city_set = set()

while True:
    url = f'http://api.aviationstack.com/v1/airports?access_key={access_key}&offset={offset}&limit={limit}'
    response = requests.get(url)
    airports = json.loads(response.text)
    for d in airports.get('data'):
        airport_set.add(d.get('airport_name'))
        city_set.add(d.get('city_iata_code'))
    if offset >= airports['pagination']['total']:
        break
    offset += limit

In [ ]:
airport = [i for i in airport_set]
city = [i for i in city_set]
print(airport[:5])
print(len(airport))
print('------')
print(city[:5])
print(len(city))

['Wilmington Railway', 'Momeik', 'Victoria Falls', 'Istanbul Airport', 'Mainz Finthen Airport']
6454
------
['KKP', 'YCB', 'OSN', 'JLS', 'MQZ']
6198


## iata -> city

In [ ]:
iata_city = {}
offset = 0

while True:
    url = f'http://api.aviationstack.com/v1/cities?access_key={access_key}&offset={offset}&limit={limit}'
    ctr_response = requests.get(url)
    data = json.loads(ctr_response.text)
    for ct in data.get('data'):
        if ct['iata_code'] not in iata_city.keys():
            iata_city[ct['iata_code']] = {'city_name': ct['city_name'], 'country': ct['country_iso2']}
    if offset >= data['pagination']['total']:
        break
    offset += limit

print(len(iata_city))

9370


In [ ]:
pprint.pprint(dict(islice(iata_city.items(), 5)))

{'AAA': {'city_name': 'Anaa', 'country': 'PF'},
 'AAB': {'city_name': 'Arrabury', 'country': 'AU'},
 'AAC': {'city_name': 'El Arish', 'country': 'EG'},
 'AAE': {'city_name': 'Annaba', 'country': 'DZ'},
 'AAF': {'city_name': 'Apalachicola', 'country': 'US'}}


## iata -> country

In [ ]:
iso2_country = {}
offset = 0

while True:
    url = f'http://api.aviationstack.com/v1/countries?access_key={access_key}&offset={offset}&limit={limit}'
    response = requests.get(url)
    data = json.loads(response.text)
    for ct in data.get('data'):
        if ct['country_iso2'] not in iso2_country.keys():
            iso2_country[ct['country_iso2']] = {'country_name': ct['country_name'], 'currency': ct['currency_name']}
    if offset >= data['pagination']['total']:
        break
    offset += limit

print(len(iso2_country))

252


In [ ]:
pprint.pprint(dict(islice(iso2_country.items(), 5)))

{'AD': {'country_name': 'Andorra', 'currency': 'Euro'},
 'AE': {'country_name': 'United Arab Emirates', 'currency': 'Dirham'},
 'AF': {'country_name': 'Afghanistan', 'currency': 'Afghani'},
 'AG': {'country_name': 'Antigua and Barbuda', 'currency': 'Dollar'},
 'AI': {'country_name': 'Anguilla', 'currency': 'Dollar'}}


### to_df

In [ ]:
city_country = []
for city_code, info in iata_city.items():
    country_info = iso2_country.get(info['country'], {'country_name':'', 'currency':''})
    if city_code is not None:
        city_country.append({
            'city_code': city_code,
            'city_name': info['city_name'],
            'country_code': info['country'],
            'country_name': country_info['country_name'],
            'currency': country_info['currency']
        })

city_df = pd.DataFrame(city_country)
print(len(city_df))
city_df.head(10)

9369


,city_code,city_name,country_code,country_name,currency
0,AAA,Anaa,PF,French Polynesia,Franc
1,AAB,Arrabury,AU,Australia,Dollar
2,AAC,El Arish,EG,Egypt,Pound
3,AAE,Annaba,DZ,Algeria,Dinar
4,AAF,Apalachicola,US,United States,Dollar
5,AAG,Arapoti,BR,Brazil,Real
6,AAH,Aachen,DE,Germany,Euro
7,AAI,Arraias,BR,Brazil,Real
8,AAJ,Awaradam,SR,Suriname,Dollar
9,AAK,Aranuka,KI,Kiribati,Dollar
